In [1]:
import tensorflow as tf
import os

In [3]:
sess = tf.InteractiveSession()

#### build a toy model

In [4]:
x = tf.placeholder(shape=(None,), dtype=tf.float32, name='input_x')

In [5]:
coef = tf.constant(10., dtype=tf.float32, name='constant_coef')

In [6]:
y = tf.reduce_sum(tf.multiply(coef, x), name='output_y')

In [7]:
output = sess.run(y, feed_dict={x: [1, 2, 3]})

In [8]:
output

60.0

#### create a `builder` with given path, it can not exist

In [10]:
export_path_base = "/Users/matt.meng/tf_test/"
model_version = 1
export_path = os.path.join(
    tf.compat.as_bytes(export_path_base),
    tf.compat.as_bytes(str(model_version)))
print 'Exporting trained model to', export_path
builder = tf.saved_model.builder.SavedModelBuilder(export_path)

Exporting trained model to /Users/matt.meng/tf_test/1


#### create a tensor_info object for each input and output

In [11]:
tensor_info_x = tf.saved_model.utils.build_tensor_info(x)
tensor_info_y = tf.saved_model.utils.build_tensor_info(y)

#### create the prediction signature object

In [12]:
prediction_signature = (
    tf.saved_model.signature_def_utils.build_signature_def(
        inputs={'input_x': tensor_info_x},
        outputs={'output_y': tensor_info_y},
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))

In [14]:
type(prediction_signature)

tensorflow.core.protobuf.meta_graph_pb2.SignatureDef

In [15]:
legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')

#### add the sigatureDef into graph meta data

In [16]:
builder.add_meta_graph_and_variables(
    sess, [tf.saved_model.tag_constants.SERVING],
    signature_def_map={
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
        prediction_signature,
    },
    legacy_init_op=legacy_init_op)

builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /Users/matt.meng/tf_test/1/saved_model.pb


'/Users/matt.meng/tf_test/1/saved_model.pb'

#### client side to do the prediction

In [3]:
from grpc.beta import implementations
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2

In [5]:
host, port = 'localhost', 9000
channel = implementations.insecure_channel(host, int(port))
stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)

In [19]:
request = predict_pb2.PredictRequest()
request.model_spec.name = 'test'
#request.model_spec.signature_name = 'predict_images'
request.inputs['input_x'].CopyFrom(
tf.contrib.util.make_tensor_proto([3., 3., 3.]))

In [20]:
result = stub.Predict(request, 10.0)  # 10 secs timeout

In [21]:
result

outputs {
  key: "output_y"
  value {
    dtype: DT_FLOAT
    tensor_shape {
    }
    float_val: 90.0
  }
}